In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

# Define hyperparameters
num_epochs = 5
batch_size = 32
learning_rate = 0.01


In [32]:
# Define data preprocessing
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_val = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the dataset
train_dataset = torchvision.datasets.ImageFolder(root='../Datasets/vegetable dataset/Vegetable Images/test', transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = torchvision.datasets.ImageFolder(root='../Datasets/vegetable dataset/Vegetable Images/train', transform=transform_val)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [33]:
# Load the pre-trained ResNet model
resnet = models.resnet18(pretrained=True)
# resnet = resnet.cuda()
# Replace the final fully connected layer
# num_ftrs = resnet.fc.in_features
# resnet.fc = nn.Linear(num_ftrs, 3)


In [34]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=learning_rate)


In [35]:
# Train the model
for epoch in range(num_epochs):
    train_loss = 0.0
    train_total = 0
    train_correct = 0
    
    for images, labels in train_loader:
        # Move tensors to the GPU if available
        # images = images.cuda()
        # labels = labels.cuda()
        
        # Forward pass
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track the training loss and accuracy
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    
    # Calculate the validation accuracy
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            # Move tensors to the GPU if available
            outputs = resnet(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    print("Epoch: {}, trainLoss: {}, testLoss: {}, trainAcc:{}, valAcc:{}".format(epoch,train_loss,val_loss,train_correct/train_total,val_correct/val_total))
            
            

KeyboardInterrupt: 